# Luther - Data Cleaning and Merging Dataframes

This is part 2 of the Luther Project. In part 1, I've created 6 different dataframes that will be merged and cleaned in this notebook. The final merged dataframe "merged2.pkl" will be used in the last notebook "03 - Luther - Linear Regression" to develop a final linear regression model for predicting movie ticket sales on opening week (as well as opening gross adjusted to ticket price).

In [1]:
import pickle
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import seaborn as sns
import datetime

In [2]:
# Import all the dataframes
mojog_df = pd.read_pickle('data/mojog_df.pkl')
unemp_df = pd.read_pickle("data/unemp_df.pkl")
youtube_df = pd.read_pickle('data/youtube_df.pkl')
omdb_df = pd.read_pickle('data/omdb_df.pkl')
director_df = pd.read_pickle('data/director_df.pkl')
actor_df = pd.read_pickle('data/actor_df.pkl')

In [3]:
#clean mojo df
mojog_df.opening = mojog_df.opening.replace("\$","", regex = True).replace(",","", regex = True).str.strip()
mojog_df.opening = pd.to_numeric(mojog_df.opening)
mojog_df.tot_gross = mojog_df.tot_gross.replace("\$","", regex = True).replace(",","", regex = True).str.strip()
mojog_df.tot_gross = pd.to_numeric(mojog_df.tot_gross)
mojog_df.theaters = mojog_df.theaters.replace(",","", regex = True).str.strip()
mojog_df.theaters = pd.to_numeric(mojog_df.theaters)
mojog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 9 columns):
movie_name    500 non-null object
movie         500 non-null object
opening       500 non-null int64
date_time     500 non-null datetime64[ns]
theaters      500 non-null int64
tot_gross     500 non-null int64
actors        458 non-null object
directors     460 non-null object
gtrend        500 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 39.1+ KB


In [4]:
# Clean youtube df 
# select youtube df relevant data
youtube_df = youtube_df[["movie_name", "viewCount", "commentCount", "dislikeCount", "likeCount"]]

#replace nulls with 0
youtube_df = youtube_df.fillna(0)
# convert numeric str values to int
youtube_df.viewCount = pd.to_numeric(youtube_df.viewCount)
youtube_df.dislikeCount = pd.to_numeric(youtube_df.dislikeCount)
youtube_df.commentCount = pd.to_numeric(youtube_df.commentCount)
youtube_df.likeCount = pd.to_numeric(youtube_df.likeCount)
youtube_df = youtube_df.rename(columns ={"viewCount":"Yviews", "commentCount": "Ycomments", 
                                         "dislikeCount": "Ydislikes", "likeCount":"Ylikes"})
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
movie_name    500 non-null object
Yviews        500 non-null int64
Ycomments     500 non-null int64
Ydislikes     500 non-null int64
Ylikes        500 non-null int64
dtypes: int64(4), object(1)
memory usage: 19.6+ KB


In [5]:
# Clean omdb data
omdb_df.imdb = pd.to_numeric(omdb_df.imdb.replace("/10","", regex = True).str.strip())
omdb_df.metacritic = pd.to_numeric(omdb_df.metacritic.replace("/100","", regex = True).str.strip())
omdb_df.rotten_tomatoes = pd.to_numeric(omdb_df.rotten_tomatoes.replace("%","", regex = True).str.strip())
omdb_df.runtime = pd.to_numeric(omdb_df.runtime.replace("min","", regex = True).str.strip())
omdb_df.year = pd.to_numeric(omdb_df.year)
# Remove 11 movies that have been wrongly identified on omdb (wrong year)
year = [2013,2014,2015,2016,2017]
omdb_df = omdb_df[omdb_df.year.isin(year)]
omdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469 entries, 0 to 479
Data columns (total 7 columns):
imdb               469 non-null float64
metacritic         469 non-null int64
movie              469 non-null object
rated              469 non-null object
rotten_tomatoes    469 non-null int64
runtime            469 non-null int64
year               469 non-null int64
dtypes: float64(1), int64(4), object(2)
memory usage: 29.3+ KB


In [6]:
# Clean directors data
# Convert str to numeric
director_df.dir_agross = pd.to_numeric(director_df.dir_agross.replace("\$","", regex = True).str.strip())
director_df.dir_gross = pd.to_numeric(director_df.dir_gross.replace("\$","", regex = True)
                                      .replace(",","", regex = True).str.strip())
director_df.dir_nmovies = pd.to_numeric(director_df.dir_nmovies)
director_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
dir_agross     500 non-null float64
dir_gross      500 non-null float64
dir_nmovies    500 non-null int64
director       500 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 15.7+ KB


In [7]:
# Clean directors data
# Convert str to numeric
actor_df.act_agross = pd.to_numeric(actor_df.act_agross.replace("\$", "", regex = True)
                                    .replace(",", "", regex = True).str.strip())
actor_df.act_gross = pd.to_numeric(actor_df.act_gross.replace("\$", "", regex = True)
                                      .replace(",", "", regex = True).str.strip())
actor_df.act_nmovies = pd.to_numeric(actor_df.act_nmovies)
actor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
act_agross     500 non-null float64
act_gross      500 non-null float64
act_nmovies    500 non-null int64
actor          500 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 15.7+ KB


In [8]:
actor_df.head()

,act_agross,act_gross,act_nmovies,actor
0,70.5,5149.1,73,Samuel L. Jackson
1,118.2,4963.8,42,Harrison Ford
2,96.0,4605.6,48,Tom Hanks
3,71.8,4522.2,63,Morgan Freeman
4,166.7,4333.5,26,Andy Serkis


In [10]:
mojog_df.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,gtrend
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],0.456
1,Iron Man 3,Iron Man 3,174144585,2013-05-03,4253,409013994,"[Robert Downey, Jr., Gwyneth Paltrow, Don Chea...",[Shane Black],0.792
2,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",0.124
3,Despicable Me 2,Despicable Me 2,83517315,2013-07-03,3997,368061265,"[Steve Carell, Kristen Wiig, Benjamin Bratt, R...","[Pierre Coffin, Chris Renaud]",0.706
4,Man of Steel,Man of Steel,116619362,2013-06-14,4207,291045518,"[Henry Cavill, Amy Adams, Michael Shannon, Kev...",[Zack Snyder],0.713


In [11]:
# Create a new column unemp_date, by dropping the datetime day to 1
# This is done in order to merge the unemployment monthly data to the movies release dates
#datetime.date(2008,1,1).replace(day=5)
mojog_df["unemp_date"] = mojog_df.date_time.apply(lambda x: x.replace(day=1))
mojog_df = pd.merge(mojog_df,unemp_df,on="unemp_date",how="inner")
mojog_df = mojog_df.drop(["unemp_date"],1)
mojog_df.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,gtrend,unemp_rate
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],0.456,6.9
1,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",0.124,6.9
2,Thor: The Dark World,Thor: The Dark World,85737841,2013-11-08,3841,206362140,"[Chris Hemsworth, Natalie Portman, Tom Hiddles...",[Alan Taylor],0.474,6.9
3,The Best Man Holiday,The Best Man Holiday,30107555,2013-11-15,2024,70525195,"[Morris Chestnut, Regina Hall, Terrence Howard...",[Malcolm D. Lee],0.220,6.9
4,Last Vegas,Last Vegas,16334566,2013-11-01,3065,63914167,"[Michael Douglas, Robert DeNiro, Morgan Freema...",[Jon Turteltaub],0.187,6.9


# Merge the dataframes

In [12]:
# Merge mojo and youtube
merged = pd.DataFrame.merge(mojog_df, youtube_df,on='movie_name', how = 'inner')
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 14 columns):
movie_name    500 non-null object
movie         500 non-null object
opening       500 non-null int64
date_time     500 non-null datetime64[ns]
theaters      500 non-null int64
tot_gross     500 non-null int64
actors        458 non-null object
directors     460 non-null object
gtrend        500 non-null float64
unemp_rate    500 non-null float64
Yviews        500 non-null int64
Ycomments     500 non-null int64
Ydislikes     500 non-null int64
Ylikes        500 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(7), object(4)
memory usage: 58.6+ KB


In [13]:
# Merge omdb
merged2 = pd.DataFrame.merge(merged,omdb_df,on="movie",how="inner")
merged2.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,gtrend,unemp_rate,Yviews,Ycomments,Ydislikes,Ylikes,imdb,metacritic,rated,rotten_tomatoes,runtime,year
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],0.456,6.9,12389287,9493,1385,52030,7.5,76,PG-13,89,146,2013
1,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",0.124,6.9,34267342,7458,8664,54157,7.5,74,PG,90,102,2013
2,Thor: The Dark World,Thor: The Dark World,85737841,2013-11-08,3841,206362140,"[Chris Hemsworth, Natalie Portman, Tom Hiddles...",[Alan Taylor],0.474,6.9,17863773,11763,2486,65159,7.0,54,PG-13,66,112,2013
3,The Best Man Holiday,The Best Man Holiday,30107555,2013-11-15,2024,70525195,"[Morris Chestnut, Regina Hall, Terrence Howard...",[Malcolm D. Lee],0.220,6.9,2064143,781,326,5425,6.7,59,R,70,123,2013
4,Last Vegas,Last Vegas,16334566,2013-11-01,3065,63914167,"[Michael Douglas, Robert DeNiro, Morgan Freema...",[Jon Turteltaub],0.187,6.9,2046856,252,234,2747,6.6,48,PG-13,45,105,2013


In [14]:
# Adjust the youtube views based on google trends data
merged2["Yviews_adj"] = round(merged2.Yviews * merged2.gtrend)
merged2["Ylikes_adj"] = round(merged2.Ylikes * merged2.gtrend)
merged2["Ydis_adj"] = round(merged2.Ydislikes * merged2.gtrend)
merged2["Ycom_adj"] = round(merged2.Ycomments * merged2.gtrend)
#merged2["Yviews_adj2"] = round(merged2.Yviews * merged2.gtrend)
merged2 = merged2.drop(["gtrend","Yviews","Ycomments","Ylikes","Ydislikes"],1)
#merged2 = merged2.drop(["gtrend","gtrend2"],1)
merged2.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,unemp_rate,imdb,metacritic,rated,rotten_tomatoes,runtime,year,Yviews_adj,Ylikes_adj,Ydis_adj,Ycom_adj
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],6.9,7.5,76,PG-13,89,146,2013,5649515.0,23726.0,632.0,4329.0
1,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",6.9,7.5,74,PG,90,102,2013,4249150.0,6715.0,1074.0,925.0
2,Thor: The Dark World,Thor: The Dark World,85737841,2013-11-08,3841,206362140,"[Chris Hemsworth, Natalie Portman, Tom Hiddles...",[Alan Taylor],6.9,7.0,54,PG-13,66,112,2013,8467428.0,30885.0,1178.0,5576.0
3,The Best Man Holiday,The Best Man Holiday,30107555,2013-11-15,2024,70525195,"[Morris Chestnut, Regina Hall, Terrence Howard...",[Malcolm D. Lee],6.9,6.7,59,R,70,123,2013,454111.0,1194.0,72.0,172.0
4,Last Vegas,Last Vegas,16334566,2013-11-01,3065,63914167,"[Michael Douglas, Robert DeNiro, Morgan Freema...",[Jon Turteltaub],6.9,6.6,48,PG-13,45,105,2013,382762.0,514.0,44.0,47.0
5,Ender's Game,Ender's Game,27017351,2013-11-01,3407,61737191,"[Harrison Ford, Abigail Breslin, Hailee Steinf...",[],6.9,6.7,51,PG-13,61,114,2013,1673684.0,4379.0,288.0,1414.0
6,Free Birds,Free Birds,15805237,2013-11-01,3736,55750480,"[Woody Harrelson, Owen Wilson, Dan Fogler]",[Jimmy Hayward],6.9,5.9,38,PG,19,91,2013,386277.0,945.0,77.0,130.0
7,Philomena,Philomena,128435,2013-11-22,4,37709979,"[Steve Coogan, Judi Dench]",[Stephen Frears],6.9,7.6,77,PG-13,92,98,2013,31939.0,87.0,5.0,10.0
8,Delivery Man,Delivery Man,7944977,2013-11-22,3036,30664106,"[Vince Vaughn, Chris Pratt]",[Ken Scott],6.9,6.4,44,PG-13,39,105,2013,593039.0,1384.0,65.0,215.0
9,Dallas Buyers Club,Dallas Buyers Club,260865,2013-11-01,9,27298285,"[Matthew McConaughey, Jared Leto, Jennifer Gar...",[Jean-Marc Vallee],6.9,8.0,84,R,93,117,2013,86449.0,215.0,9.0,40.0


In [15]:
# First create a dataframe for year and corresponding movie ticket
year = [2013,2014,2015,2016,2017]
tick = [8.13,8.17,8.43,8.65,8.97]
adjuster = pd.DataFrame({"year":year,"tick":tick})
# Assign ticket price adjuster column to omdb through merge
merged2 = pd.merge(merged2,adjuster,on="year",how="inner")

# Create new column for merged
est_tick = (merged2.opening // merged2.tick).astype("int64")
merged2.insert(loc=3, column='est_tick', value=est_tick)
merged2 = merged2.rename(columns={"opening":"op_gross"})

In [16]:
merged2 = merged2.fillna(0)
merged2.head()

# Generate a metric for actor and directors

#### actors
I took the max total gross or total # movies featured for the actor/director for each given movie.

In [18]:
import numpy as np

act_list = []
for i in range(len(merged2)):
    #for each of the actors in a given movie, sum up their total gross & movies
    gross_list = []
    nmovies_list = []
    if (merged2.actors[i] == 0) or (merged2.actors[i] == []):
        act_gross, act_nmovies = 0, 0
    else:
        actor_count = len(merged2.actors[i])
        for actoriter in merged2.actors[i]:
            if actor_df[actor_df.actor == actoriter].empty:
                actor_count -= 1
            #Subtract the movie total gross (movie i want to predict) from actor total gross
            else:
                gross_list.append(actor_df[actor_df.actor == actoriter].act_gross.iloc[0] - \
                merged2.tot_gross[i]/(1e6))
                nmovies_list.append(actor_df[actor_df.actor == actoriter].act_nmovies.iloc[0])
    #take the sum and average over number of actors featured
    if (actor_count <= 0) or (merged2.actors[i] == []) or (merged2.actors[i] == 0):
        act_gross, act_nmovies = 0, 0
    else:
        act_gross = round(max(gross_list),1)
        act_nmovies = round(max(nmovies_list),1)
    act_dict = {'movie': merged2.movie[i], 'act_gross':act_gross, 
                'act_nmovies': act_nmovies}
    act_list.append(act_dict)
actor_metric = pd.DataFrame(act_list)

#### directors

In [19]:
import numpy as np

dir_list = []
for i in range(len(merged2)):
    #for each of the actors in a given movie, sum up their total gross & movies
    gross_list = []
    nmovies_list = []
    if (merged2.directors[i] == 0) or (merged2.directors[i] == []):
        dir_gross, dir_nmovies = 0, 0
    else:
        director_count = len(merged2.directors[i])
        for directoriter in merged2.directors[i]:
            if director_df[director_df.director == directoriter].empty:
                director_count -= 1
            #Subtract the movie total gross (movie i want to predict) from actor total gross
            else:
                gross_list.append(director_df[director_df.director == directoriter].
                                  dir_gross.iloc[0] - \
                merged2.tot_gross[i]/(1e6))
                nmovies_list.append(director_df[director_df.director == directoriter]
                                    .dir_nmovies.iloc[0])
    #take the sum and average over number of actors featured
    if (director_count <= 0) or (merged2.directors[i] == []) or (merged2.directors[i] == 0):
        dir_gross, dir_nmovies = 0, 0
    else:
        dir_gross = round(max(gross_list),1)
        dir_nmovies = round(max(nmovies_list),1)
    dir_dict = {'movie': merged2.movie[i], 'dir_gross':dir_gross, 
                'dir_nmovies': dir_nmovies}
    dir_list.append(dir_dict)
director_metric = pd.DataFrame(dir_list)

# Merge the director & actor metrics to df

In [20]:
merged2 = pd.merge(merged2, actor_metric, on="movie", how="inner")
merged2 = pd.merge(merged2, director_metric, on="movie", how="inner")
merged2 = merged2.drop(["actors","directors"],1)
merged2.head()

,movie_name,movie,op_gross,est_tick,date_time,theaters,tot_gross,unemp_rate,imdb,metacritic,...,year,Yviews_adj,Ylikes_adj,Ydis_adj,Ycom_adj,tick,act_gross,act_nmovies,dir_gross,dir_nmovies
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,19443331,2013-11-22,4163,424668047,6.9,7.5,76,...,2013,5649515.0,23726.0,632.0,4329.0,8.13,3333.4,56,1056.5,7
1,Frozen,Frozen,243390,29937,2013-11-22,1,400738009,6.9,7.5,74,...,2013,4249150.0,6715.0,1074.0,925.0,8.13,1774.2,19,230.0,3
2,Thor: The Dark World,Thor: The Dark World,85737841,10545859,2013-11-08,3841,206362140,6.9,7.0,54,...,2013,8467428.0,30885.0,1178.0,5576.0,8.13,2968.6,57,89.7,2
3,The Best Man Holiday,The Best Man Holiday,30107555,3703266,2013-11-15,2024,70525195,6.9,6.7,59,...,2013,454111.0,1194.0,72.0,172.0,8.13,1328.5,34,346.3,9
4,Last Vegas,Last Vegas,16334566,2009171,2013-11-01,3065,63914167,6.9,6.6,48,...,2013,382762.0,514.0,44.0,47.0,8.13,4458.3,81,843.5,10


### Save the merged dataframe

In [422]:
merged2.to_pickle("data/merged_df.pkl")

In [302]:
len(merged2[(merged2.dir_nmovies == 0)])# | (merged2.act_gross == 0)])

118